<a href="https://colab.research.google.com/github/mostafa-ja/graph-tutorial/blob/master/Illegal_Bitcoin_Transactions(GCN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Install Kaggle API.
!pip install kaggle
# Run the following code to configure the path to “kaggle.json”
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle datasets download -d ellipticco/elliptic-data-set
!unzip elliptic-data-set.zip

elliptic-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  elliptic-data-set.zip
replace elliptic_bitcoin_dataset/elliptic_txs_classes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [48]:
!pip install torch_geometric

In [49]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'axes.facecolor':'dimgrey', 'grid.color':'lightgrey'})

import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn.functional as F
import torch.nn as nn
#import torch_scatter
from torch_geometric.data import Data
print(torch.__version__)

# # The PyG built-in GCNConv
# from torch_geometric.nn import GCNConv
from torch_geometric.nn.conv import MessagePassing
import torch_geometric.transforms as T
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax, degree

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
import scipy.sparse as scsp
from sklearn.cluster import KMeans
import copy

2.0.1+cu118


In [50]:
path = '/content'

edges = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
classes = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
features = pd.read_csv(path + "/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

In [51]:
features.head(2)

,0,1,2,3,4,5,6,7,8,9
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162097
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.162112


In [52]:
classes.head(2)

,txId,class
0,230425980,unknown
1,5530458,unknown


In [53]:
edges.head(2)

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [54]:
# rename features
rename_dict = dict(
    zip(
        range(0, 167),
        ["txId", "time_step"]
        + [i for i in range(2, 168)]
    )
)
features.rename(columns=rename_dict, inplace=True)

In [55]:
features.head()

,txId,time_step,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [56]:
# rename the classes to ints that can be handled by pytorch as labels
classes["class"] = classes["class"].replace(
    {"unknown": -1,  # unlabeled nodes
     "2": 0,  # labeled licit nodes
     #"1": 1,  # labeled illicit nodes
    }
).astype(int)

In [57]:
# Merge features and labels
df = features.merge(classes, how="left", left_on="txId", right_on="txId")
df.head()

,txId,time_step,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,class
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,-1
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,-1
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,-1
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,0
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,-1


In [58]:
class_column = df.pop('class')


In [59]:
df.insert(1, 'class', class_column)

In [60]:
df.head()

,txId,class,time_step,2,3,4,5,6,7,8,...,157,158,159,160,161,162,163,164,165,166
0,230425980,-1,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,-1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,-1,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,-1,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [61]:
# Sort the DataFrame and reset index
#df.sort_values("txId", inplace=True)
#df.reset_index(drop=True, inplace=True)

# Check the length of df and classes
if len(df) != len(classes):
    print("Error: Lengths of df and classes do not match.")

# Drop unclassified and isolated nodes
classified_nodes = set(classes[classes["class"] != -1]["txId"].values)
classified_edges = edges[(edges["txId1"].isin(classified_nodes)) & (edges["txId2"].isin(classified_nodes))].copy()
non_isolated_nodes = set(classified_edges["txId1"].values).union(classified_edges["txId2"].values)
classified_df = df[df["txId"].isin(non_isolated_nodes)].copy()


In [65]:
classified_df.reset_index(drop=True, inplace=True)
classified_df.head()

,txId,class,time_step,2,3,4,5,6,7,8,...,157,158,159,160,161,162,163,164,165,166
0,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
1,232029206,0,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,...,-0.577099,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792
2,232344069,0,1,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
3,27553029,0,1,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.539735,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
4,3881097,0,1,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,...,-0.577099,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984


In [67]:
classified_df.info

<bound method DataFrame.info of             txId  class  time_step         2         3         4          5  \
0      232438397      0          1  0.163054  1.963790 -0.646376  12.409294   
1      232029206      0          1 -0.005027  0.578941 -0.091383   4.380281   
2      232344069      0          1 -0.147852 -0.184668 -1.201369  -0.121970   
3       27553029      0          1 -0.151357 -0.184668 -1.201369  -0.121970   
4        3881097      0          1 -0.172306 -0.184668 -1.201369   0.028105   
...          ...    ...        ...       ...       ...       ...        ...   
35869   80329479      0         49 -0.159293 -0.037276  1.018602  -0.121970   
35870  158406298      0         49 -0.172962 -0.126566  1.018602  -0.121970   
35871  158375075      1         49 -0.170412 -0.078164  1.018602   0.028105   
35872  147478192      0         49 -0.093732 -0.116160  1.018602  -0.121970   
35873  158375402      1         49 -0.172014 -0.078182  1.018602   0.028105   

              6    

In [66]:
classified_edges.reset_index(drop=True, inplace=True)
classified_edges.head()

,txId1,txId2
0,232344069,27553029
1,3881097,232457116
2,232051089,232470704
3,230473487,7089694
4,231182296,14660781


In [68]:
classified_edges.info

<bound method DataFrame.info of            txId1      txId2
0      232344069   27553029
1        3881097  232457116
2      232051089  232470704
3      230473487    7089694
4      231182296   14660781
...          ...        ...
36619  194020062   47521535
36620  158574502  109383451
36621  158594124  157631640
36622  157631640   21644119
36623  158365409  157930723

[36624 rows x 2 columns]>

In [72]:
classified_edges.values

array([[232344069,  27553029],
       [  3881097, 232457116],
       [232051089, 232470704],
       ...,
       [158594124, 157631640],
       [157631640,  21644119],
       [158365409, 157930723]])

In [69]:
features =  torch.FloatTensor(classified_df.iloc[:, 2:].to_numpy())
labels = classified_df.iloc[:, 1].to_numpy()
labels = torch.LongTensor(labels.reshape(len(labels),))
nodes_id = classified_df.iloc[:, 0].to_numpy()

In [71]:
features.shape

torch.Size([35874, 166])

In [80]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# train_size = 0.6
# test_size = 0.3
# val_size = 0.1

In [85]:
# Create a dictionary that maps nodeId to index in the dataframe.
id2idx = {}
for i in range(classified_df.shape[0]):
    id2idx[classified_df.iloc[i, 0]] = i

# Construct edge_index with same node indexing as in features and labels
edge_idx = np.zeros((2, classified_edges.shape[0]), dtype = np.int64)
for index in range(classified_edges.shape[0]):
    node1 = id2idx[classified_edges.iloc[index, 0]]
    node2 = id2idx[classified_edges.iloc[index, 1]]
    edge_idx[:, index] = [node1, node2]
edge_index = torch.LongTensor(edge_idx)

train_index, test_index = train_test_split(np.arange(labels.shape[0]),
                                            test_size = 0.4,
                                            random_state = 42)
val_index, test_index = train_test_split(test_index,
                                          test_size = 0.75,
                                          random_state = 42)

# Construct the training, validation, test set and store in a dictionary, 'data'.
data = Data(x = features, edge_index = edge_index, y = labels).to(device)


In [104]:
split_idx = {'train': train_index, 'val': val_index, 'test': test_index}

In [96]:
print('train_size = ',train_index.shape[0]/labels.shape[0])
print('test_size = ',test_index.shape[0]/labels.shape[0])
print('val_size = ',val_index.shape[0]/labels.shape[0])

train_size =  0.5999888498634108
test_size =  0.30002230027317833
val_size =  0.09998884986341083


In [97]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds = False):
        """
        Initialize a GCN model.
        Args:
            input_dim       Input dimension of node embeddings
            hidden_dim      Hidden dimension of node embeddings
            output_dim      Output dimension of node embeddings
            num_layers      The number of GCN layers
            dropout         The dropout ratio in (0, 1]
                              (dropout: the probability of an element getting zeroed)
            return_embeds   A boolean value determining whether we skip the
                              classification layer and return node embeddings
        """

        super(GCN, self).__init__()

        # Construct all convs
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList([GCNLayer(hidden_dim, hidden_dim, directed = False)
                                                        for i in range(self.num_layers-1)])

        # Construct batch normalization
        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim)
                                        for i in range(self.num_layers-1)])
        # First GCN layer
        self.convs[0] = GCNLayer(input_dim, hidden_dim, directed = False)
        # Last GCN layer
        self.last_conv = GCNLayer(hidden_dim, output_dim, directed = False)
        self.softmax = torch.nn.LogSoftmax(dim = -1)

        self.dropout = dropout
        self.return_embeds = return_embeds

    def reset_parameters(self):
        """
        Reset all learnable parameters in GCN layers and Batch Normalization
        Layers.
        """
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, edge_index):
        """
        Produce a forward propagation of GCN model. Before the last GCN layer,
        we transform the embedding (x) in the following sequence:
          x -> GCN_Layer -> Batch_Norm -> ReLU -> Dropout.
        At the last GCN layer, the following sequence is applied:
          x -> GCN Layer -> Softmax -> output.

        Args:
            x             The node embedding
            edge_index    The adjacency list of the graph

        Returns:
            out           The predictions of labels / the updated node embedding
        """
        x = torch.clone(x.detach())
        for l in range(self.num_layers - 1):
            # Unweighted graph has weight 1.
            x = self.convs[l](x, edge_index, torch.ones(edge_index.shape[1]))
            x = self.bns[l](x)
            x = F.relu(x)
            x = F.dropout(x, p = self.dropout, training = self.training)

        x = self.last_conv(x, edge_index, torch.ones(edge_index.shape[1]))
        if self.return_embeds:
            out = x
        else:
            out = self.softmax(x)

        return out

In [98]:
class GCNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, bias = True,
                 directed = False, self_loop = True, **kwargs):
        """
        Initialize a GCN layer.
        Args:
            in_channels      In-channel dimension of node embeddings
            out_channels     Out-channel dimension of node embeddings
            bias             A boolean value determining whether we add a
                                learnable bias term in linear transformation
            directed         A boolean value determining whether we use directed
                                message passing D^{-1}A or use symmetric normalized
                                adjacency matrix D^{-1/2}AD^{-1/2}
            self_loop        A boolean value determining whether we add a self-
                                loop for each node
        """
        super(GCNLayer, self).__init__(**kwargs, aggr = 'add')

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.directed = directed
        self.self_loop = self_loop

        # Define the layers needed for the message and update functions below.
        # self.lin is the linear transformation that we apply to the embedding.
        self.lin = nn.Linear(self.in_channels, self.out_channels, bias = bias)

        self.reset_parameters()

    def reset_parameters(self):
        """
        Reset all learnable parameters in the linear transformation.
        """
        self.lin.reset_parameters()

    def forward(self, x, edge_index, edge_weight):
        """
        Produce a forward propagation of GCN layer.

        Args:
            x             The node embedding
            edge_index    The (2, |E|) adjacency list of the graph
            edge_weight   The (|E|) vector specifying the edge weights in the graph
                            (for unweighted graph, edge weight is 1)

        Returns:
            An updated node embedding
        """
        # Add self-loops to the adjacency matrix.
        if self.self_loop:
            edge_index, _ = add_self_loops(edge_index, num_nodes = x.size(0))
            edge_weight = torch.cat((edge_weight, torch.ones(x.size(0))), dim = -1)

        # Apply linear transformation on node features.
        x = self.lin(x)

        # Compute normalization by updated node degree.
        if self.directed:
            row , _ = edge_index
            deg = degree(row, x.size(0), dtype = x.dtype) # only out-degree
            deg_inv = deg.pow(-1)
            deg_inv[deg_inv == float('inf')] = 0
            norm = deg_inv[row]
        else:
            row, col = edge_index
            deg = degree(col, x.size(0), dtype = x.dtype)
            deg_inv_sqrt = deg.pow(-0.5)
            deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
            norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return self.propagate(edge_index, x = (x, x), norm = norm, edge_weight = edge_weight)

    def message(self, x_j, edge_weight, norm):
        """
        Send the message of the neighboring node (i.e., x_j) to the source node (i.e., x_i).

        Args:
            x_j           The embedding of the neighboring node of source node x_i
            edge_weight   The edge weight of certain edge
            norm          Normalization constant determined by self.directed

        Returns:
            A message sending from the neighboring node to the source node
        """
        return norm.view(-1, 1) * x_j * edge_weight.view(-1, 1)

In [99]:
def train_transd(model, data, train_idx, optimizer, loss_fn):
    """
    Train the model by using the given optimizer and loss_fn.

    Args:
        model       The GCN model
        data        The Data object that stores x, edge_index, and labels
        train_idx   The node indices in the training set
        optimizer   The optimizer
        loss_fn     The loss function

    Returns
        The prediction loss by the given loss function
    """
    model.train()
    loss = 0

    optimizer.zero_grad()
    train_slice = model.forward(data.x, data.edge_index)[train_idx]
    train_label = data.y[train_idx]
    loss = loss_fn(train_slice, train_label)

    loss.backward()
    optimizer.step()
    return loss.item()

In [106]:
@torch.no_grad()
def test_transd(model, data, split_idx, save_model_results=False):
    """
    Test the model by using the given split_idx.

    Args:
        model                 The GCN model
        data                  The Data object that stores x, edge_index, and labels
        split_idx             A dictionary that stores node indices for three sets
        save_model_results    A boolean determining whether we save the model results

    Returns
        The accuracy and auc-roc score of training, validation, and test set
    """

    model.eval()
    # The output of model on all data
    out = model.forward(data.x, data.edge_index)

    train_index = split_idx['train']
    val_index = split_idx['val']
    test_index = split_idx['test']

    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = classification_report(torch.unsqueeze(data.y[train_index], -1),
                                      y_pred[train_index], zero_division=0)
    valid_acc = classification_report(torch.unsqueeze(data.y[val_index], -1),
                                      y_pred[val_index], zero_division=0)
    valid_accuracy = classification_report(torch.unsqueeze(data.y[val_index], -1),
                                      y_pred[val_index], output_dict=True,
                                      zero_division=0)['accuracy']
    test_acc = classification_report(torch.unsqueeze(data.y[test_index], -1),
                                     y_pred[test_index], zero_division=0)
    train_auc_roc = roc_auc_score(torch.unsqueeze(data.y[train_index], -1),
                                      y_pred[train_index])
    val_auc_roc = roc_auc_score(torch.unsqueeze(data.y[val_index], -1),
                                      y_pred[val_index])
    test_auc_roc = roc_auc_score(torch.unsqueeze(data.y[test_index], -1),
                                      y_pred[test_index])

    if save_model_results:
        print ("Saving Model Predictions")

        data = {}
        data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

        df = pd.DataFrame(data=data)
        # Save locally as csv
        df.to_csv('gcn_transd.csv', sep=',', index=False)

    return train_acc, valid_acc, test_acc, \
           train_auc_roc, val_auc_roc, test_auc_roc

In [100]:
args = {
    'device': device,
    'num_layers': 2,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 50,
    'label_weight': torch.Tensor([0.5, 0.5])
}
args

{'device': 'cpu',
 'num_layers': 2,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 50,
 'label_weight': tensor([0.5000, 0.5000])}

In [101]:
model = GCN(data.x.shape[1], args['hidden_dim'],
            2, args['num_layers'], args['dropout']).to(device)

In [107]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.NLLLoss(weight=args['label_weight'])

best_model = None
best_valid_auc = 0
best_result = None
losses = []

for epoch in range(1, 1 + args["epochs"]):
    # train with random split
    loss = train_transd(model, data, split_idx['train'], optimizer, loss_fn)
    losses.append(loss)
    result = test_transd(model, data, split_idx)
    train_acc, val_acc, test_acc, train_auc, val_auc, test_auc = result
    if val_auc > best_valid_auc:
        best_valid_auc = val_auc
        best_model = copy.deepcopy(model)
        best_result = [train_acc, val_acc, test_acc, train_auc, val_auc, test_auc]

    print('Epoch: {:02},'.format(epoch),
          'Loss:{:.4f}'.format(loss),
          'Train:\n{}\n'.format(train_acc),
          'Train_auc_roc: {}'.format(train_auc),
          '\n\n'
          'Valid:\n{}\n'.format(val_acc),
          'Val_auc_roc: {}'.format(val_auc),
          '\n\n'
          'Test:\n{}\n'.format(test_acc),
          'Test_auc_roc: {}'.format(test_auc),
          '\n'
          )


Epoch: 01, Loss:0.2554 Train:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     20040
           1       0.00      0.00      0.00      1484

    accuracy                           0.93     21524
   macro avg       0.47      0.50      0.48     21524
weighted avg       0.87      0.93      0.90     21524

 Train_auc_roc: 0.5 

Valid:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3357
           1       0.00      0.00      0.00       230

    accuracy                           0.94      3587
   macro avg       0.47      0.50      0.48      3587
weighted avg       0.88      0.94      0.90      3587

 Val_auc_roc: 0.5 

Test:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      9993
           1       0.00      0.00      0.00       770

    accuracy                           0.93     10763
   macro avg       0.46      0.50      0.48